In [ ]:
%pip install pscript

In [2]:
import base64
base64.encodestring = base64.encodebytes
base64.decodestring = base64.decodebytes
from pscript import py2js

In [ ]:
%pip install elasticsearch==7.17

In [31]:
from elasticsearch import Elasticsearch
import os
import re
import pandas as pd
import string

In [32]:
es = Elasticsearch("http://localhost:9200", timeout=60)

In [33]:
os.chdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection")
files = os.listdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection")
files.remove("readme")

In [35]:
# function to get DOCNO keys for dataframe from ap89 files

def get_docnums():
    test_keys = []
    for i in files[:364]:
        current = open(str("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection/"+i), encoding = "latin-1")
        text = current.read()
        keys = re.findall("<DOCNO>(.*)</DOCNO>", text)
        for j in keys:
            test_keys.append(j)
    return test_keys

#test_keys = get_docnums()
#print(len(test_keys))

In [36]:
# function to merge multiple TEXTs of the same DOCNO

def find_anom(text):
    between = re.findall("</TEXT>(.*?)<TEXT>", text)
    anoms = [x for x in between if "<DOCNO>" not in x]
    for n in anoms:
        if n in text:
            text = text.replace(n, "")
    return text

In [37]:
# function to remove information not needed in parsing

def remove_others(text):
    fileid = re.findall("<FILEID>(.+?)</FILEID>", text)
    first = re.findall("<FIRST>(.+?)</FIRST>", text)
    second = re.findall("<SECOND>(.+?)</SECOND>", text)
    head = re.findall("<HEAD>(.+?)</HEAD>", text)
    byline = re.findall("<BYLINE>(.+?)</BYLINE>", text)
    dateline = re.findall("<DATELINE>(.+?)</DATELINE>", text)
    note = re.findall("<NOTE>(.+?)</NOTE>", text)
    removal = fileid + first + second + head + byline + dateline + note
    for x in removal:
        if x in text:
            text = text.replace(x, "")
    sections = ["<DOC>", "</DOC>", "<FILEID>", "</FILEID>", "<FIRST>", "</FIRST>", "<SECOND>", "</SECOND>", 
    "<HEAD>", "</HEAD>", "<BYLINE>", "</BYLINE>", "<DATELINE>", "</DATELINE>", "<NOTE>", "</NOTE>", 
    "</TEXT><TEXT>", "</TEXT> <TEXT>", "</TEXT>  <TEXT>", "</TEXT>   <TEXT>", 
    "</TEXT> files a0511-a0514-a0517-a0518; inserts new 5thgraf, In another, with leading indicators report; edits pvs 5th graf fortransition<TEXT>", 
    "</TEXT><UNK>Eds: For release 11 a.m. EDT, time set by source.</UNK><TEXT>", "</TEXT> For release 8:01 p.m. EDT<TEXT>", 
    "</TEXT> ^Consumer Advocates, Insurers Join Forces<TEXT>", "</TEXT>By s<TEXT>", "</TEXT>s<TEXT>"]
    for y in sections:
        if y in text:
            text = text.replace(y, "")
    return text


In [38]:
# get TEXT values for data frame
def get_texts():
    test_values = []
    for i in files[:364]:
        current = open(str("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection/"+i), encoding = 'latin-1')
        text = current.read()
        text = ''.join(text.splitlines())
        text = find_anom(text)
        text = remove_others(text)
        values = re.findall("<TEXT>(.*?)</TEXT>", text)
        for j in values:
            test_values.append(j)
    return test_values

#test_values = get_texts()
#print(len(test_values))


In [39]:
# function to get stopwords from txt file

def get_sw():
    with open("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/stoplist.txt") as sw:
        stop_words = sw.read().split()
    return stop_words

In [40]:
# function to format TEXT to remove punctuation,remove stop words, and make lowercase for 

def format_text():
    formatted_values = []
    stop_words = get_sw()
    test_values = get_texts()

    for text in test_values:
        new_string = re.sub("-", " ", text)
        new_string = re.sub("\n", " ", new_string)
        new_string = new_string.translate(str.maketrans('', '', string.punctuation))
        new_string = ' '.join([word for word in new_string.split() if word not in stop_words])
        formatted_values.append(new_string.lower())
    return formatted_values

#formatted_values = format_text()
#print(formatted_values[1])

In [41]:
# create dataframe from <DOCNO> key list and <TEXT> value list
def create_docno_text_df():
    test_keys = get_docnums()
    formatted_values = format_text()
    
    ap89_data = pd.DataFrame()
    ap89_data["DOCNO"] = test_keys
    ap89_data["TEXT"] = formatted_values
    return ap89_data

#ap89_data = create_docno_text_df()
#print(ap89_data.iloc[:3])

In [85]:
# function to create elastic search index
def create_index():
    mapping = {
        "mappings": {
            "properties": {
                "TEXT": {
                    "type": "text",
                    "store": True,
                    "term_vector": "with_positions_offsets_payloads",
                    "analyzer": "standard"
                    }
                 }
            }
        }
    es.indices.create(index="ap89_index4", body=mapping)

In [ ]:
# function to write dataframe DOCNO and TEXT to elastic search index
def write_index():
    ap89_data = create_docno_text_df()
    create_index()

    for i, row in ap89_data.iterrows():
        doc_num = row["DOCNO"]
        doc = {
            "TEXT": row["TEXT"]
        }  
        es.index(index="ap89_index4", id=doc_num, body=doc)

write_index()

In [87]:
# use to check index

es.indices.refresh(index="ap89_index4")
es.cat.count(index="ap89_index", format="json")

[{'epoch': '1675637564', 'timestamp': '22:52:44', 'count': '84678'}]